In [0]:
import urllib.request

urls = [
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    "https://datasets.imdbws.com/title.principals.tsv.gz",
    "https://datasets.imdbws.com/title.crew.tsv.gz",
    "https://datasets.imdbws.com/title.akas.tsv.gz",
    "https://datasets.imdbws.com/title.episode.tsv.gz",
    "https://datasets.imdbws.com/name.basics.tsv.gz"
]

for url in urls:
    filename = "/Volumes/workspace/imdb_project/filestore/" + url.split("/")[-1]
    urllib.request.urlretrieve(url, filename)

print("Download complete!")

In [0]:
import gzip
import shutil
import os

src_path = "/Volumes/workspace/imdb_project/filestore/"
dst_path = "/Volumes/workspace/imdb_project/bronze/"

# create bronze folder if not exists
dbutils.fs.mkdirs(dst_path)

files = [f.name for f in dbutils.fs.ls(src_path) if f.name.endswith(".gz")]

for file in files:
    gz_file = src_path + file
    tsv_file = dst_path + file.replace(".gz", "")

    with gzip.open(gz_file.replace("dbfs:", ""), 'rb') as f_in:
        with open(tsv_file.replace("dbfs:", ""), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

print("Unzipping complete!")


In [0]:
%pip install ydata-profiling


In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS imdb_project")


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS imdb_project_bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS imdb_project_silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS imdb_project_rejects")


In [0]:
spark.sql("USE imdb_project_bronze")


In [0]:
spark.sql("SELECT current_catalog(), current_database()").show()


In [0]:
from pyspark.sql import functions as F

path = "/Volumes/workspace/imdb_project/bronze/name.basics.tsv"

df = (
    spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("name.basics.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.name_basics")

display(spark.table("workspace.imdb_project_bronze.name_basics").limit(10))


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.name_basics").show()


In [0]:
sample_df = spark.table("workspace.imdb_project_bronze.name_basics") \
                 .limit(100000) \
                 .toPandas()


In [0]:
from ydata_profiling import ProfileReport

profile = ProfileReport(sample_df, 
                        title="IMDb – name.basics Profiling Report",
                        explorative=True)

profile.to_file("/Workspace/Shared/imdb_name_basics_profile.html")


In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.basics.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.basics.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_basics")

display(spark.table("workspace.imdb_project_bronze.title_basics").limit(10))


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_basics").show()

In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.akas.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.akas.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_akas")

display(spark.table("workspace.imdb_project_bronze.title_akas").limit(10))


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_akas").show()

In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.crew.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.crew.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_crew")

display(spark.table("workspace.imdb_project_bronze.title_crew").limit(10))


In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.episode.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.episode.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_episode")

display(spark.table("workspace.imdb_project_bronze.title_episode").limit(10))


In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.principals.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.principals.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_principals")

display(spark.table("workspace.imdb_project_bronze.title_principals").limit(10))


In [0]:
path = "/Volumes/workspace/imdb_project/bronze/title.ratings.tsv"

df = (spark.read
        .option("sep", "\t")
        .option("header", True)
        .option("nullValue", "\\N")
        .csv(path)
)

df = df.withColumn("source_file", F.lit("title.ratings.tsv")) \
       .withColumn("load_datetime", F.current_timestamp())

df.write.mode("overwrite").saveAsTable("workspace.imdb_project_bronze.title_ratings")

display(spark.table("workspace.imdb_project_bronze.title_ratings").limit(10))


In [0]:
from ydata_profiling import ProfileReport

# Dictionary of IMDb Bronze tables
datasets = {
    "title_basics": "workspace.imdb_project_bronze.title_basics",
    "title_akas": "workspace.imdb_project_bronze.title_akas",
    "title_crew": "workspace.imdb_project_bronze.title_crew",
    "title_episode": "workspace.imdb_project_bronze.title_episode",
    "title_principals": "workspace.imdb_project_bronze.title_principals",
    "title_ratings": "workspace.imdb_project_bronze.title_ratings"
}

for name, table in datasets.items():
    print(f"Profiling {name} ...")

    # Sample data to avoid overwhelming cluster
    pdf = (
        spark.table(table)
        .limit(100000)
        .toPandas()
    )

    profile = ProfileReport(
        pdf,
        title=f"IMDb {name} Profiling Report",
        explorative=True
    )

    # Save report
    output_path = f"/Workspace/Shared/{name}_profile.html"
    profile.to_file(output_path)

    print(f"✔ Saved: {output_path}\n")

print("ALL REPORTS GENERATED!")


In [0]:
from pyspark.sql import functions as F

bronze = spark.table("workspace.imdb_project_bronze.name_basics")

# 1. Rows with null nconst -> reject
reject_null_key = bronze.filter(F.col("nconst").isNull()) \
    .withColumn("reject_reason", F.lit("nconst is NULL"))

reject_null_key.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.name_basics_null_nconst"
)

# 2. Cast years, mark invalid years
clean = bronze.filter(F.col("nconst").isNotNull()) \
    .withColumn("birthYear_int", F.col("birthYear").cast("int")) \
    .withColumn("deathYear_int", F.col("deathYear").cast("int")) \
    .withColumn("invalid_birth_year",
                (F.col("birthYear_int") < 1850) | (F.col("birthYear_int") > 2030)) \
    .withColumn("invalid_death_year",
                (F.col("deathYear_int") < 1850) | (F.col("deathYear_int") > 2035))

# 3. Send rows with crazy years to rejects
reject_years = clean.filter(F.col("invalid_birth_year") | F.col("invalid_death_year")) \
    .withColumn("reject_reason", F.lit("Birth/Death year out of range"))

reject_years.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.name_basics_bad_years"
)

# 4. Keep only valid rows, drop helper flags
silver = clean.filter(~(F.col("invalid_birth_year") | F.col("invalid_death_year"))) \
    .drop("invalid_birth_year", "invalid_death_year") \
    .withColumn("birthYear_clean", F.col("birthYear_int")) \
    .withColumn("deathYear_clean", F.col("deathYear_int")) \
    .drop("birthYear_int", "deathYear_int")

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.name_basics"
)

print("Silver name_basics rows:",
      spark.table("workspace.imdb_project_silver.name_basics").count())


In [0]:
from pyspark.sql import functions as F

bronze = spark.table("workspace.imdb_project_bronze.title_basics")

# Reject null tconst
reject_null_key = bronze.filter(F.col("tconst").isNull()) \
    .withColumn("reject_reason", F.lit("tconst is NULL"))

reject_null_key.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_basics_null_tconst"
)

clean = bronze.filter(F.col("tconst").isNotNull()) \
    .withColumn(
        "startYear_int",
        F.when(F.col("startYear").rlike("^[0-9]+$"), F.col("startYear").cast("int")).otherwise(None)
    ) \
    .withColumn(
        "endYear_int",
        F.when(F.col("endYear").rlike("^[0-9]+$"), F.col("endYear").cast("int")).otherwise(None)
    ) \
    .withColumn(
        "runtimeMinutes_int",
        F.when(F.col("runtimeMinutes").rlike("^[0-9]+$"), F.col("runtimeMinutes").cast("int")).otherwise(None)
    ) \
    .withColumn(
        "isAdult_bool",
        F.when(F.col("isAdult").rlike("^[0-9]+$"), F.col("isAdult").cast("int")).otherwise(0)
    ) \
    .withColumn("genres_array",
        F.when(F.col("genres").isNotNull(), F.split(F.col("genres"), ","))
         .otherwise(F.array().cast("array<string>"))
    ) \
    .withColumn(
        "invalid_year",
        (F.col("startYear_int") < 1850) | (F.col("startYear_int") > 2030)
    ) \
    .withColumn(
        "invalid_runtime",
        F.col("runtimeMinutes_int") < 0
    )

# Reject rows with invalid year/runtime
reject_bad = clean.filter(F.col("invalid_year") | F.col("invalid_runtime")) \
    .withColumn("reject_reason", F.lit("Invalid year or runtime"))

reject_bad.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_basics_invalid_values"
)

# Keep only valid rows
silver = clean.filter(~(F.col("invalid_year") | F.col("invalid_runtime"))) \
    .drop("invalid_year", "invalid_runtime")

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_basics"
)

print("Silver title_basics rows:",
      spark.table("workspace.imdb_project_silver.title_basics").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_basics").show()

In [0]:
from pyspark.sql import functions as F

bronze = spark.table("workspace.imdb_project_bronze.title_akas")

# Reject rows without titleId
reject_null = bronze.filter(F.col("titleId").isNull()) \
    .withColumn("reject_reason", F.lit("titleId is NULL"))

reject_null.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_akas_null_titleId"
)

silver = bronze.filter(F.col("titleId").isNotNull()) \
    .withColumn("isOriginalTitle_bool",
        F.when(F.col("isOriginalTitle") == "1", 1)
         .when(F.col("isOriginalTitle") == "0", 0)
         .otherwise(None)
    ) \
    .withColumn("ordering_int",
        F.when(F.col("ordering").rlike("^[0-9]+$"), F.col("ordering").cast("int"))
        .otherwise(None)
    )

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_akas"
)

print("Silver title_akas rows:",
      spark.table("workspace.imdb_project_silver.title_akas").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_akas").show()

In [0]:
from pyspark.sql import functions as F

bronze = spark.table("workspace.imdb_project_bronze.title_crew")

# Reject rows where primary key is missing
reject_null = bronze.filter(F.col("tconst").isNull()) \
    .withColumn("reject_reason", F.lit("tconst is NULL"))

reject_null.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_crew_null_tconst"
)

silver = bronze.filter(F.col("tconst").isNotNull()) \
    .withColumn(
        "directors_array",
        F.when(F.col("directors").isNotNull(), F.split(F.col("directors"), ","))
         .otherwise(F.array().cast("array<string>"))
    ) \
    .withColumn(
        "writers_array",
        F.when(F.col("writers").isNotNull(), F.split(F.col("writers"), ","))
         .otherwise(F.array().cast("array<string>"))
    )

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_crew"
)

print("Silver title_crew rows:",
      spark.table("workspace.imdb_project_silver.title_crew").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_crew").show()

In [0]:
bronze = spark.table("workspace.imdb_project_bronze.title_episode")

# Reject NULL key
reject_null = bronze.filter(F.col("tconst").isNull()) \
    .withColumn("reject_reason", F.lit("tconst is NULL"))

reject_null.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_episode_null_tconst"
)

silver = bronze.filter(F.col("tconst").isNotNull()) \
    .withColumn(
        "seasonNumber_int",
        F.when(F.col("seasonNumber").rlike("^[0-9]+$"),
               F.col("seasonNumber").cast("int"))
         .otherwise(None)
    ) \
    .withColumn(
        "episodeNumber_int",
        F.when(F.col("episodeNumber").rlike("^[0-9]+$"),
               F.col("episodeNumber").cast("int"))
         .otherwise(None)
    )

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_episode"
)

print("Silver title_episode rows:",
      spark.table("workspace.imdb_project_silver.title_episode").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_episode").show()

In [0]:
bronze = spark.table("workspace.imdb_project_bronze.title_principals")

# Reject if either primary key is missing
reject_null = bronze.filter(
    F.col("tconst").isNull() | F.col("nconst").isNull()
).withColumn("reject_reason", F.lit("tconst or nconst is NULL"))

reject_null.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_principals_null_keys"
)

silver = bronze.filter(
    F.col("tconst").isNotNull() & F.col("nconst").isNotNull()
).withColumn(
    "characters_clean",
    F.regexp_replace("characters", r'["\[\]]', "")  # removes ["",[]]
)

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_principals"
)

print("Silver title_principals rows:",
      spark.table("workspace.imdb_project_silver.title_principals").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_principals").show()

In [0]:
bronze = spark.table("workspace.imdb_project_bronze.title_ratings")

# Reject NULL tconst
reject_null = bronze.filter(F.col("tconst").isNull()) \
    .withColumn("reject_reason", F.lit("tconst is NULL"))

reject_null.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_ratings_null_tconst"
)

clean = bronze.filter(F.col("tconst").isNotNull()) \
    .withColumn("averageRating_float", F.col("averageRating").cast("double")) \
    .withColumn("numVotes_int", F.col("numVotes").cast("int")) \
    .withColumn(
        "invalid_record",
        (F.col("averageRating_float") < 0) |
        (F.col("averageRating_float") > 10) |
        (F.col("numVotes_int") < 0)
    )

reject_bad = clean.filter(F.col("invalid_record")) \
    .withColumn("reject_reason", F.lit("Invalid rating or numVotes"))

reject_bad.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_rejects.title_ratings_invalid_values"
)

silver = clean.filter(~F.col("invalid_record")).drop("invalid_record")

silver.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_silver.title_ratings"
)

print("Silver title_ratings rows:",
      spark.table("workspace.imdb_project_silver.title_ratings").count())


In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.imdb_project_bronze.title_ratings").show()

In [0]:
tables = ["name_basics","title_basics","title_akas",
          "title_crew","title_episode","title_principals","title_ratings"]

for t in tables:
    bronze = spark.table(f"workspace.imdb_project_bronze.{t}").count()
    silver = spark.table(f"workspace.imdb_project_silver.{t}").count()
    print(f"{t}: Bronze={bronze}, Silver={silver}, Dropped={bronze-silver}")


In [0]:
%sql
SHOW TABLES IN workspace.imdb_project_silver;


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS imdb_project_gold")


In [0]:
silver = "workspace.imdb_project_silver"

df = spark.table(f"{silver}.name_basics") \
    .select(
        "nconst",
        "primaryName",
        "birthYear_clean",
        "deathYear_clean",
        "primaryProfession",
        "knownForTitles"
    ) \
    .dropDuplicates(["nconst"])

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.dim_persons"
)


In [0]:
df = spark.table(f"{silver}.title_basics") \
    .select(
        "tconst",
        "titleType",
        "primaryTitle",
        "originalTitle",
        "isAdult_bool",
        "startYear_int",
        "endYear_int",
        "runtimeMinutes_int"
    )

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.dim_titles"
)


In [0]:
df = spark.table(f"{silver}.title_akas") \
    .select("region") \
    .where("region IS NOT NULL") \
    .dropDuplicates()

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.dim_regions"
)


In [0]:
df = spark.table(f"{silver}.title_akas") \
    .select("language") \
    .where("language IS NOT NULL") \
    .dropDuplicates()

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.dim_languages"
)


In [0]:
from pyspark.sql import functions as F

df = spark.table(f"{silver}.title_basics") \
    .select(F.explode("genres_array").alias("genre")) \
    .where("genre IS NOT NULL") \
    .dropDuplicates()

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.dim_genres"
)


In [0]:
df = spark.table(f"{silver}.title_basics") \
    .select("tconst", F.explode("genres_array").alias("genre"))

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.bridge_title_genres"
)


In [0]:
df = spark.table(f"{silver}.name_basics") \
    .select(
        "nconst",
        F.explode(F.split("primaryProfession", ",")).alias("profession")
    )

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.bridge_person_profession"
)


In [0]:
df = spark.table(f"{silver}.title_akas") \
    .select("titleId", "region")

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.bridge_title_region"
)


In [0]:
df = spark.table(f"{silver}.title_akas") \
    .select("titleId", "language")

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.bridge_title_language"
)


In [0]:
df = spark.table(f"{silver}.title_ratings") \
    .select("tconst", "averageRating_float", "numVotes_int")

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.fact_ratings"
)


In [0]:
df_titles = spark.table(f"{silver}.title_basics") \
    .withColumnRenamed("load_datetime", "title_load_dt")

df_ratings = spark.table(f"{silver}.title_ratings") \
    .withColumnRenamed("load_datetime", "ratings_load_dt")

df = df_titles.join(df_ratings, "tconst", "left").drop("source_file")

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.fact_movies"
)


In [0]:
df = spark.table(f"{silver}.title_principals") \
    .select(
        "tconst", "nconst", "category", "job", "characters_clean"
    )

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.fact_principals"
)


In [0]:
df = spark.table(f"{silver}.title_episode") \
    .select(
        "tconst",
        "parentTconst",
        "seasonNumber_int",
        "episodeNumber_int"
    )

df.write.mode("overwrite").saveAsTable(
    "workspace.imdb_project_gold.fact_episodes"
)


In [0]:
%sql
SHOW TABLES IN workspace.imdb_project_gold;


In [0]:
gold_schema = "workspace.imdb_project_gold"

tables = [
    "dim_persons",
    "dim_titles",
    "dim_regions",
    "dim_languages",
    "dim_genres",
    "bridge_title_genres",
    "bridge_person_profession",
    "bridge_title_region",
    "bridge_title_language",
    "fact_ratings",
    "fact_movies",
    "fact_principals",
    "fact_episodes"
]

print("📊 GOLD LAYER ROW COUNTS\n")

for t in tables:
    try:
        count = spark.table(f"{gold_schema}.{t}").count()
        print(f"{t:30} → {count}")
    except Exception as e:
        print(f"{t:30} → ERROR: {str(e)}")
